# Chapter 11-03: Triton 커널 프로그래밍

## 학습 목표
- OpenAI Triton의 **Block-level 프로그래밍 모델**과 CUDA 스레드 모델의 차이를 이해한다
- Triton의 **Block Pointer**와 **타일 매핑** 패러다임으로 Fused Softmax 커널을 설계한다
- **Online Softmax** 알고리즘을 수식으로 도출하고, 단일 패스에서 수치적으로 안정적인 Softmax 계산 원리를 증명한다
- Triton 스타일의 Fused Softmax를 NumPy로 시뮬레이션하고, 표준 방식 대비 메모리 접근 횟수를 비교한다

## 목차
1. [수학적 기초: Online Softmax와 Kernel Fusion](#1.-수학적-기초)
2. [CUDA 모델 vs Triton 모델 비교](#2.-CUDA-vs-Triton)
3. [표준 Softmax의 메모리 비효율 분석](#3.-표준-Softmax-분석)
4. [Online Softmax 알고리즘 구현](#4.-Online-Softmax-구현)
5. [Fused Softmax 시뮬레이션](#5.-Fused-Softmax-시뮬레이션)
6. [Triton 스타일 FlashAttention 핵심 원리](#6.-FlashAttention-원리)
7. [정리 및 연습 문제](#7.-정리)

## 1. 수학적 기초

### 표준 Softmax의 3-pass 메모리 접근

입력 벡터 $x \in \mathbb{R}^S$에 대한 표준 Softmax 계산:

1. **Pass 1 (Read)**: $m = \max_i x_i$ — 수치 안정화용 최댓값
2. **Pass 2 (Read + Write)**: $e_i = \exp(x_i - m)$, $d = \sum_i e_i$
3. **Pass 3 (Read + Write)**: $y_i = e_i / d$

**총 Global Memory 접근**: $5S$ bytes (3회 읽기 + 2회 쓰기)

### Online Softmax: 단일 패스로 통합!

새 원소 $x_j$를 처리할 때 기존 통계량을 업데이트합니다:

$$m_j = \max(m_{j-1}, x_j)$$

$$d_j = d_{j-1} \cdot \exp(m_{j-1} - m_j) + \exp(x_j - m_j)$$

이렇게 하면 **Pass 1과 2가 합쳐져** 2-pass (또는 블록 단위 1-pass)로 구현 가능!

**수학적 등가성 증명**:

$$d_S = \sum_{i=1}^S \exp(x_i - m_S)$$

왜냐하면 $d_j$의 재귀식에서 $m_j$가 단조 증가하므로, 이전 항들을 $\exp(m_{old} - m_{new})$로 보정하면 항상 올바른 분모가 유지됩니다.

---

### 🐣 초등학생을 위한 친절 설명!

#### 🔄 Kernel Fusion이 왜 중요한가요?

> 여러 요리(Add, Softmax, Dropout)를 각각 따로 만들면  
> 매 요리마다 재료(데이터)를 창고(메모리)에서 꺼내야 해요.
>
> **Fusion**: 세 요리를 한 번에 해결! 재료를 한 번만 가져오고, 한 번에 세 가지를 다 처리.
> → 창고(메모리) 왕복 횟수가 1/3로 줄어드는 셈!

#### 📊 Online Softmax가 뭔가요?

> 일반 Softmax는 먼저 전체 점수를 다 보고(Pass 1), 그 다음 exp를 계산(Pass 2)해요.  
> Online Softmax는 한 번 훑으면서 **동시에** 최댓값과 합산을 업데이트해요!
> 마치 물건 가격을 보면서 동시에 최고가와 합계를 계산하는 것처럼요.

### 📝 연습 문제

#### 문제 1: 메모리 접근 비교

시퀀스 길이 $S=4096$, FP16에서 표준 Softmax(3-pass)와 Fused Softmax(2-pass+Write 1회)의  
HBM 읽기 바이트 수를 각각 계산하라.

<details>
<summary>💡 풀이 확인</summary>

FP16 = 2 bytes/element, $S=4096$

- **표준 (3-pass)**: 읽기 3회 + 쓰기 2회 = $5 \times 4096 \times 2 = 40.96$ KB
- **Fused (1-pass)**: 읽기 1회 + 쓰기 1회 = $2 \times 4096 \times 2 = 16.38$ KB
- 절약: **2.5배** HBM 대역폭 절감

(FlashAttention에서는 QK^T 행렬 자체도 HBM 안 씀 → 훨씬 큰 절감 효과)
</details>

#### 문제 2: Online Softmax 정확성 검증

$x = [3.0, 1.0, 4.0, 1.5, 2.0]$에 대해 Online Softmax를 손으로 단계별 계산하고  
표준 Softmax와 결과를 비교하라. (계산 지원 허용)

<details>
<summary>💡 풀이 확인</summary>

표준: $m=4.0$, $d = e^{-1}+e^{-3}+e^{0}+e^{-2.5}+e^{-2} \approx 0.368+0.050+1.0+0.082+0.135 = 1.635$  
→ $y \approx [0.225, 0.031, 0.612, 0.050, 0.083]$

Online: $m_1=3.0$, $d_1=1.0$; $m_2=3.0$, $d_2=1+e^{-2}=1.135$;  
$m_3=4.0$, $d_3=1.135\times e^{-1}+1=1.418$; ... (최종 결과 동일)

**수치적으로 동일**한 결과 → Online Softmax 올바름 ✅
</details>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

## 2. CUDA vs Triton

In [ ]:
# ---------------------------------------------------
# CUDA 모델 vs Triton Block 모델 비교 설명
# ---------------------------------------------------

comparison = '''
┌─────────────────────────────────┬────────────────────────────────────┐
│       CUDA 스레드 모델          │      Triton Block 모델              │
├─────────────────────────────────┼────────────────────────────────────┤
│ 프로그래밍 단위: 개별 스레드    │ 프로그래밍 단위: 스레드 블록       │
│                                 │                                    │
│ threadIdx.x / blockIdx.x 직접  │ tl.program_id(axis=0)              │
│ 다뤄야 함                       │ (블록 ID만 신경 씀)                │
│                                 │                                    │
│ Shared Memory를 명시적으로 관리 │ Shared Memory 자동 관리!           │
│ → __shared__, bank conflict     │ → tl.load/store, mask 기반         │
│                                 │                                    │
│ 타이진: 개발자가 직접 설계      │ 타이링: 블록 크기로 자동 결정       │
│                                 │                                    │
│ 언어: CUDA C++                  │ 언어: Python                       │
│ 컴파일: nvcc                    │ 컴파일: @triton.jit 자동            │
├─────────────────────────────────┼────────────────────────────────────┤
│ 난이도: ★★★★★                 │ 난이도: ★★★                        │
│ 유연성: ★★★★★                 │ 유연성: ★★★★                       │
│ 최적화: ★★★★★                 │ 최적화: ★★★★ (~90% of cuBLAS)      │
└─────────────────────────────────┴────────────────────────────────────┘
'''
print(comparison)

triton_softmax_pseudocode = '''
@triton.jit
def softmax_kernel(X_ptr, Y_ptr, stride, n_cols, BLOCK: tl.constexpr):
    # ① 프로그램(블록) ID로 처리할 행 결정
    row = tl.program_id(0)
    row_start = row * stride

    # ② 열 오프셋 계산 (벡터화)
    cols = tl.arange(0, BLOCK)
    mask = cols < n_cols

    # ③ 데이터 로드 (Shared Memory 자동 관리)
    x = tl.load(X_ptr + row_start + cols, mask=mask, other=-float(\'inf\'))

    # ④ 수치 안정화 Softmax (1-pass!)
    x_max = tl.max(x, axis=0)          # Pass 1: 최댓값
    x = x - x_max                      # 빼기로 안정화
    exp_x = tl.exp(x)                  # Pass 2: exp
    sum_exp = tl.sum(exp_x, axis=0)    # 합산 (Pass 2와 합침!)
    y = exp_x / sum_exp                # 정규화

    # ⑤ 결과 저장 (1회 쓰기)
    tl.store(Y_ptr + row_start + cols, y, mask=mask)
'''
print("Triton Fused Softmax 커널 의사코드:")
print(triton_softmax_pseudocode)

## 4. Online Softmax 구현

In [ ]:
# ---------------------------------------------------
# Online Softmax 완전 구현 및 검증
# ---------------------------------------------------

def softmax_standard(x):
    """표준 Softmax (수치 안정화 포함, 3-pass)"""
    # Pass 1: 최댓값 계산
    m = np.max(x, axis=-1, keepdims=True)
    # Pass 2: exp 계산 + 합산
    e = np.exp(x - m)
    d = np.sum(e, axis=-1, keepdims=True)
    # Pass 3: 정규화
    return e / d


def online_softmax_elementwise(x):
    """
    Online Softmax: 원소 하나씩 처리하며 m, d 동시 업데이트.
    플래시어텐션(FlashAttention)의 핵심 알고리즘.
    
    재귀 관계:
      m_j = max(m_{j-1}, x_j)
      d_j = d_{j-1} * exp(m_{j-1} - m_j) + exp(x_j - m_j)
    """
    m = -np.inf
    d = 0.0
    exp_x = np.zeros_like(x)

    # 1-pass: m과 d를 동시에 업데이트
    for i, xi in enumerate(x):
        m_new = max(m, xi)
        d = d * np.exp(m - m_new) + np.exp(xi - m_new)
        m = m_new
        exp_x[i] = np.exp(xi - m)  # 현 시점 m 기준 exp

    # 최종 정규화 (exp 값들도 현재 m 기준으로 보정됨)
    # 주의: 위에서 저장된 exp_x는 중간 m 기준이므로 최종 m으로 재조정 필요
    exp_x_final = np.exp(x - m)  # 최종 m으로 통일
    return exp_x_final / d


def online_softmax_blocked(x, block_size=8):
    """
    Block 단위 Online Softmax (Triton/FlashAttention 방식).
    블록을 처리하며 m, d를 누적.
    """
    S = len(x)
    m = -np.inf
    d = 0.0

    # 블록 단위 1-pass
    for start in range(0, S, block_size):
        block = x[start:start + block_size]
        m_block = np.max(block)
        m_new = max(m, m_block)
        # 기존 d를 새 m으로 보정 + 새 블록 기여
        d = d * np.exp(m - m_new) + np.sum(np.exp(block - m_new))
        m = m_new

    # 단일 Normalization pass
    return np.exp(x - m) / d


# 검증
x_test = np.array([3.0, 1.0, 4.0, 1.5, 2.0, 5.0, 0.5, 3.5], dtype=np.float64)

y_std    = softmax_standard(x_test)
y_online = online_softmax_elementwise(x_test)
y_block  = online_softmax_blocked(x_test, block_size=4)

print("[Online Softmax 검증]")
print(f"입력: {x_test}")
print(f"\n표준:    {y_std.round(6)}")
print(f"Online:  {y_online.round(6)}")
print(f"Block:   {y_block.round(6)}")
print()
print(f"표준 vs Online 최대 오차: {np.abs(y_std - y_online).max():.2e} {'✅' if np.allclose(y_std, y_online, atol=1e-10) else '❌'}")
print(f"표준 vs Block  최대 오차: {np.abs(y_std - y_block).max():.2e} {'✅' if np.allclose(y_std, y_block, atol=1e-10) else '❌'}")
print(f"\n합산 검증: sum(softmax) = {y_std.sum():.10f} ≈ 1")
print("\n결론: Online Softmax는 표준 Softmax와 수학적으로 완전히 동일!")
print("     단일 또는 블록 단위 패스로 메모리 접근 최소화 달성")

## 5. Fused Softmax 시뮬레이션

In [ ]:
# ---------------------------------------------------
# Fused Softmax 성능 이득 정량화
# ---------------------------------------------------

import time

def count_hbm_accesses(S, method='standard'):
    """HBM 접근 원소 수 계산 (FP16 = 2bytes/원소)"""
    if method == 'standard':
        return 5 * S  # 3읽기 + 2쓰기
    elif method == 'fused':
        return 2 * S  # 1읽기 + 1쓰기

seq_lengths = [512, 1024, 2048, 4096, 8192, 16384, 32768]

print(f"{'시퀀스 길이':>12} | {'표준 HBM (KB)':>14} | {'Fused HBM (KB)':>15} | {'절약':>8}")
print("-" * 58)
for S in seq_lengths:
    std_bytes  = count_hbm_accesses(S, 'standard') * 2 / 1024
    fused_bytes = count_hbm_accesses(S, 'fused') * 2 / 1024
    savings = (1 - fused_bytes / std_bytes) * 100
    print(f"{S:>12} | {std_bytes:>14.1f} | {fused_bytes:>15.1f} | {savings:>7.0f}%")

# 시뮬레이션 성능 테스트
S = 8192
batch = 64
X = np.random.randn(batch, S).astype(np.float32)

# 표준 (Python에서 3-pass 시뮬레이션)
n_runs = 100
t0 = time.perf_counter()
for _ in range(n_runs):
    Y_std = softmax_standard(X)
t_std = (time.perf_counter() - t0) / n_runs * 1000

# Fused (2-pass 시뮬레이션)
t0 = time.perf_counter()
for _ in range(n_runs):
    m = np.max(X, axis=-1, keepdims=True)   # Pass 1
    e = np.exp(X - m)                        # Pass 2 part 1
    d = np.sum(e, axis=-1, keepdims=True)    # Pass 2 part 2 (합산)
    Y_fused = e / d                           # 정규화 (Pass 2 내)
t_fused = (time.perf_counter() - t0) / n_runs * 1000

print(f"\n실행 시간 비교 (batch={batch}, S={S}, NumPy):")
print(f"  표준 3-pass:  {t_std:.2f} ms")
print(f"  Fused 2-pass: {t_fused:.2f} ms")
print(f"  등가 검증: {'✅' if np.allclose(Y_std, Y_fused, atol=1e-5) else '❌'}")
print(f"\n(실제 GPU에서는 HBM 접근 감소로 2~3배 빠름, 여기서는 NumPy CPU 측정)")

## 6. FlashAttention 원리

In [ ]:
# ---------------------------------------------------
# FlashAttention의 핵심: Fused Online Attention
# Online Softmax + Attention 합쳐서 O(S) HBM!
# ---------------------------------------------------

def flash_attention_naive_reference(Q, K, V, scale=None):
    """
    표준 Attention (메모리 비효율 버전).
    S×S의 Attention Score 행렬을 HBM에 쓰고 읽음.
    메모리: O(S²)
    """
    S, d = Q.shape[-2], Q.shape[-1]
    if scale is None:
        scale = 1.0 / np.sqrt(d)

    scores = Q @ K.transpose(0, 2, 1) * scale  # (B, S, S) — HBM에 저장!
    attn   = softmax_standard(scores)            # (B, S, S)
    output = attn @ V                            # (B, S, d)
    return output, attn


def flash_attention_block(Q, K, V, block_size=4, scale=None):
    """
    FlashAttention 핵심 알고리즘 (블록 단위 Online Softmax).
    S×S 행렬을 HBM에 저장하지 않음 → 메모리 O(S)
    """
    B, S, d = Q.shape
    if scale is None:
        scale = 1.0 / np.sqrt(d)

    O = np.zeros_like(Q)  # 출력 누적
    m = np.full((B, S, 1), -np.inf)  # 행별 최댓값
    l = np.zeros((B, S, 1))          # 행별 분모 합

    # KV 블록 단위로 처리 (HBM에서 블록씩 로드)
    for j in range(0, S, block_size):
        j_end = min(j + block_size, S)
        Kj = K[:, j:j_end, :]   # KV 블록
        Vj = V[:, j:j_end, :]

        # Q × Kj^T → (B, S, block_size) (HBM에 저장하지 않음)
        Sij = Q @ Kj.transpose(0, 2, 1) * scale

        # Online Softmax 업데이트
        mij  = np.max(Sij, axis=-1, keepdims=True)
        m_new = np.maximum(m, mij)

        # 기존 O, l을 새 m으로 보정 + 새 블록 기여 추가
        exp_new = np.exp(Sij - m_new)
        alpha = np.exp(m - m_new)  # 기존 항목 보정 계수

        O = alpha * O + exp_new @ Vj
        l = alpha * l + np.sum(exp_new, axis=-1, keepdims=True)
        m = m_new

    return O / l  # 최종 정규화 (1회만!)


# 검증
B, S, d = 2, 16, 8
Q = np.random.randn(B, S, d).astype(np.float64)
K = np.random.randn(B, S, d).astype(np.float64)
V = np.random.randn(B, S, d).astype(np.float64)

O_std, _   = flash_attention_naive_reference(Q, K, V)
O_flash    = flash_attention_block(Q, K, V, block_size=4)

err_fa = np.abs(O_std - O_flash).max()
print("[FlashAttention Block 검증]")
print(f"  입력: B={B}, S={S}, d={d}, block_size=4")
print(f"  최대 오차: {err_fa:.2e}")
print(f"  수학적 등가: {'✅ 완전 일치' if err_fa < 1e-10 else '❌ 불일치'}")
print()
print("  표준 Attention HBM: O(S²) = S×S Attention Score 행렬")
print(f"  Flash Attention HBM: O(S) = 블록 크기만큼만 로드 (Q,K,V 원본 제외)")
print(f"  S={S}에서 HBM 절약: {S}² = {S**2} 원소 vs {S}×{d} = {S*d} 원소 ({S//d}배↓)")

In [ ]:
# ---------------------------------------------------
# Triton / FlashAttention HBM 절약 시각화
# ---------------------------------------------------

seq_range = np.array([512, 1024, 2048, 4096, 8192])
d_model   = 64  # head dimension

# 표준 Attention HBM (S×S + Q,K,V)
standard_hbm = (seq_range**2 + 3 * seq_range * d_model) * 2  # FP16

# FlashAttention HBM (블록 단위, S×d만)
flash_hbm = 5 * seq_range * d_model * 2  # Q,K,V 읽기 + O 쓰기 + 중간

fig, axes = plt.subplots(1, 2, figsize=(13, 5))

ax1 = axes[0]
ax1.semilogy(seq_range, standard_hbm / 1e6, 'r-o', lw=2.5, ms=8, label='표준 Attention: O(S²)')
ax1.semilogy(seq_range, flash_hbm / 1e6, 'b-s', lw=2.5, ms=8, label='FlashAttention: O(S)')
ax1.fill_between(seq_range, flash_hbm/1e6, standard_hbm/1e6, alpha=0.1, color='red')
for s, st, fl in zip(seq_range, standard_hbm, flash_hbm):
    ax1.annotate(f'{int(st/fl)}×↓', (s, (st*fl)**0.5/1e6),
                fontsize=9, ha='center', color='darkred')
ax1.set_xlabel('시퀀스 길이 (S)', fontsize=11)
ax1.set_ylabel('HBM 접근량 (MB, log scale)', fontsize=11)
ax1.set_title('표준 Attention vs FlashAttention\nHBM 접근량 비교', fontweight='bold')
ax1.legend(fontsize=10); ax1.grid(True, alpha=0.3)

ax2 = axes[1]
methods = ['Softmax\n표준\n(3-pass)', 'Softmax\nFused\n(2-pass)', 'Attention\n표준\n(O(S²))', 'FlashAttn\n(O(S))']
hbm_norm = [100, 40, 100, 100*d_model/4096]  # S=4096 기준 정규화
colors_bar = ['#E53935', '#43A047', '#E53935', '#1E88E5']
bars = ax2.bar(methods, hbm_norm, color=colors_bar, alpha=0.85, edgecolor='white', lw=2)
for bar, val in zip(bars, hbm_norm):
    ax2.text(bar.get_x() + bar.get_width()/2, val + 1,
             f'{val:.0f}%', ha='center', fontsize=11, fontweight='bold')
ax2.set_ylabel('HBM 접근량 (표준 대비 %)')
ax2.set_title('Kernel Fusion 효과\n(S=4096, d=64 기준)', fontweight='bold')
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 7. 정리

### 핵심 개념 요약

| 개념 | 설명 |
|------|------|
| **Kernel Fusion** | 여러 연산을 1개 커널로 합쳐 HBM 접근 최소화 |
| **Online Softmax** | 원소를 순차 처리하며 $m$, $d$ 동시 업데이트 → 1-pass |
| **Triton Block** | CUDA 스레드 대신 블록 단위로 프로그래밍 (Python) |
| **FlashAttention** | Online Softmax를 Attention에 적용 → $O(S^2) \to O(S)$ HBM |

### Online Softmax 핵심 재귀

$$m_j = \max(m_{j-1}, x_j), \quad d_j = d_{j-1} \cdot e^{m_{j-1}-m_j} + e^{x_j - m_j}$$

### 다음 챕터 예고
**Chapter 11-04: 커널 프로파일링과 Kernel Fusion 전략** — GPU 커널 성능 측정 도구(Nsight Systems, nvprof), 병목 분석 방법론, 여러 연산을 합치는 Fusion 설계 전략을 학습한다.